# CS-VQE model data generation

Generates all of the data required to conduct CS-VQE simulations using utils.molecule_tools.construct_molecule and the CS-VQE circuit class cs_vqe.circuit.cs_vqe_circuit.

In [1]:
from utils.molecule_tools import construct_molecule
from utils.cs_vqe_tools_original import greedy_dfs
from cs_vqe.circuit import cs_vqe_circuit
import json

/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Will store matrices in sparse form


/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.optimizers',


In [2]:
with open('data/molecule_data.json', 'r') as json_file:
    molecule_data = json.load(json_file)
with open('data/model_data.json', 'r') as json_file:
    model_data = json.load(json_file)
#with open('data/anz_circ_depth.json', 'r') as json_file:
#    anz_data = json.load(json_file)
    
#speciesname_list = [mol for mol in anz_data if (anz_data[mol]['num_qubits']!=anz_data[mol]['chemaccnum'] and not anz_data[mol]['chemaccnum']==0) and mol.find('alt')==-1 and mol.find('+')==-1]
#speciesname_list

In [3]:
#model_data = {}

for speciesname in ['B+_STO-3G_SINGLET']:#speciesname_list:
    print(speciesname)
    
    atoms, coords, multiplicity, charge, basis, sym_sector = molecule_data[speciesname].values()
    molecule    = construct_molecule(atoms, coords, charge, multiplicity, basis, 
                                     excitation_threshold=None, taper=True, sym_sector=sym_sector)
    num_qubits  = molecule['num_qubits']
    ham         = molecule['hamiltonian']
    ansatze     = molecule['ansatze']
    hf_config   = molecule['hf_config']
    hf_energy   = molecule['hf_energy']
    num_tapered = molecule['num_tapered']
    true_gs_nrg = molecule['true_gs_nrg']
    #true_gs_vec = molecule['true_gs_vec']

    terms_noncon = greedy_dfs(ham, 30, criterion='size')[-1]

    print('Number of qubits:', num_qubits)
    mol_circ = cs_vqe_circuit(hamiltonian = ham,
                              terms_noncon= terms_noncon,
                              num_qubits  = num_qubits,
                              hf_config   = hf_config)

    ham_rotations  = mol_circ.ham_rotations
    noncon         = mol_circ.noncon
    truegs         = mol_circ.truegs
    G              = mol_circ.G
    A              = mol_circ.A
    X_index        = mol_circ.X_index
    X_qubit        = mol_circ.X_qubit
    cs_vqe_energy  = mol_circ.cs_vqe_energy
    cs_vqe_errors  = mol_circ.cs_vqe_errors
    chem_acc_num_q = mol_circ.chem_acc_num_q
    order          = mol_circ.order
    ham_reduced    = mol_circ.ham_reduced
    reference_state= mol_circ.reference_state()

    model_data[speciesname] = {"ham":ham,
                               "ansatze":ansatze,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "hf_energy":hf_energy,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg,
                               #"true_gs_vec":true_gs_vec,
                               "ham_rotations":ham_rotations,
                               "noncon":noncon,
                               "truegs":truegs,
                               "G":G,
                               "A":A,
                               "X_index":X_index,
                               "X_qubit":X_qubit,
                               "cs_vqe_energy":cs_vqe_energy,
                               "cs_vqe_errors":cs_vqe_errors,
                               "chem_acc_num_q":chem_acc_num_q,
                               "order":order,
                               "ham_reduced":ham_reduced,
                               "reference_state":reference_state
                                }

    print('Molecule constructed.\n')


B+_STO-3G_SINGLET
Attempting to taper 10 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, -1, 1, 1, 1), tapering successful!

Number of qubits: 5
Molecule constructed.



In [5]:
#data={}
#for k in model_data.keys():
#    data[str(k)] = model_data[k]
     
with open("data/model_data.json", "w") as outfile:
    json.dump(model_data, outfile)

In [12]:
for name, a in model_data['HF_STO-3G_SINGLET']['ansatze'].items():
    print(name, len(a))

uccsd 44
excite3 8
excite4 44
excite5 44


In [4]:
model_data['B+_STO-3G_SINGLET']

{'ham': {'IIIII': -15.660830489309033,
  'ZIIII': 0.10025489288748901,
  'IZIII': 0.10025489288748989,
  'ZZIII': 0.507698665839882,
  'ZZZII': 0.3423143361228139,
  'IIIZI': 0.19716678433985396,
  'ZIIZI': 0.23856909889297526,
  'IZIZI': 0.23856909889297498,
  'ZZIZI': -0.005353447789286306,
  'IIZZI': -0.028192859786666354,
  'ZIZZI': 0.3656910831625782,
  'IZZZI': 0.36569108316258214,
  'ZZZZI': 3.977170261095493,
  'IIIIZ': 0.19716678433985438,
  'ZIIIZ': 0.2385690988929748,
  'IZIIZ': 0.2385690988929751,
  'ZZIIZ': 0.005353447789286813,
  'IIZIZ': 0.0281928597866656,
  'ZIZIZ': 0.36569108316258014,
  'IZZIZ': 0.36569108316258053,
  'ZZZIZ': 3.9771702610954973,
  'IIIZZ': 0.8557945626523968,
  'ZZIZZ': 1.4849232954361469e-15,
  'ZZZZZ': 0.37123935915830236,
  'XIIII': 0.005353447789286756,
  'XZZZI': -0.005353447789286756,
  'XZZIZ': -0.005353447789286756,
  'XIIZZ': 0.005353447789286756,
  'IXIII': 0.005353447789286756,
  'ZXZZI': -0.005353447789286756,
  'ZXZIZ': -0.0053534477892